## Carga de librerias e incializacion


In [9]:
from preprocess import load_data
from bayes import BayesPredictor

In [10]:
FILENAME='Datos/chat.txt'
HORIZONTE=4
m=2


data=load_data(FILENAME)
print(data.head)
predictor=BayesPredictor(data["palabras"],HORIZONTE,m)

Es IOS?:  True
<bound method NDFrame.head of                                                 crudo  ...                                           palabras
0   [2/9/23 18:42:58] Mauro Freduc: Mis queridos, ...  ...  [mis, queridos, espero, se, encuentren, muy, b...
1   ‎[2/9/23 18:43:09] Mauro Freduc: ‎imagen: omit...  ...                                  [imagen, omitida]
5   [2/9/23 18:44:42] Zapa: Buena Maurito, bueno f...  ...  [buena, maurito, bueno, felicitaciones, viejo,...
6   [2/9/23 18:45:08] Mauro Freduc: Zapa querido m...  ...     [zapa, querido, muchísimas, gracias, mi, loco]
7   [2/9/23 18:45:51] Zapa: Vamos a tener que junt...  ...  [vamos, a, tener, que, juntarnos, a, comer, un...
8   [2/9/23 18:47:49] Jesus: ‎Mauro Freduc añadió ...  ...                  [mauro, freduc, añadió, a, jesus]
9   [2/9/23 18:47:57] Nico Goncalvez: ‎Mauro Fredu...  ...        [mauro, freduc, añadió, a, nico, goncalvez]
10  [2/9/23 18:46:32] German: Felicitaciones viejo...  ...          [felici

In [11]:

##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
        break

    elif palabra == ".":
        predictor.update(frase)
            
        print("----- Comenzando frase nueva -----")
        frase = []

    elif palabra == "": # acepta última palabra sugerida
        frase.append(palabra_sugerida)

    else: # escribió una palabra
        frase.append(palabra)

    if frase:
        palabra_sugerida = predictor.predict(frase,verbose=True)[0]
    
        frase_propuesta = frase.copy()
        frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")
    
        print(" ".join(frase_propuesta))



Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
[('mis', 0.0023228803716608595), ('queridos', 0.0023228803716608595), ('espero', 0.0023228803716608595), ('se', 0.0023228803716608595), ('encuentren', 0.0023228803716608595)]
 mis
[('mis', 0.0023228803716608595), ('queridos', 0.0023228803716608595), ('espero', 0.0023228803716608595), ('se', 0.0023228803716608595), ('encuentren', 0.0023228803716608595)]
 Hola mis
[('queridos', 0.0007796892304413337), ('espero', 0.0007796892304413337), ('se', 0.0007796892304413337), ('encuentren', 0.0007796892304413337), ('mis', 5.395773221047293e-06)]
 Hola mis queridos
[('espero', 0.00026170753495649877), ('se', 0.00026170753495649877), ('encuentren', 0.00026170753495649877), ('queridos', 1.8111248093875347e-06), ('muy', 1.8111248093875347e-06)]
 Hola mis queridos esp